In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [4]:
to_replace = [(constant.UNDEF_VAL, 'mean')]

augment_param_0 = {
    'degrees': [2,3],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': False
}

augment_param_1 = {
    'degrees': [2],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': True
}

augment_param_23 = {
    'degrees': [2],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': True
}

augment_param_list = [augment_param_0, augment_param_1, augment_param_23]

import time

start = time.time()


x_split, y_split, jet_num_to_idx = pp.preprocess_jet_num(x, y, to_replace, do_normalise=True, augment_param_list=augment_param_list)

end = time.time()
print(end - start)

700.8710989952087


In [5]:
def train_and_eval(data_part):

    models = []

    learning_param = {
            'lambda_': 1e-6,
            'gamma': 1e-6,
            'max_iters': 200
    }

    # lambdas = np.logspace(-4, 0, 30)
    # lambdas = [0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
    lambdas = [learning_param['lambda_']]

    acc_list = [[] for _ in lambdas]

    for i, lambda_ in enumerate(lambdas):
        learning_param['lambda_'] = lambda_

        for idx in range(len(data_part)):
            x_tr, y_tr = data_part[idx]['train']
            nb_features = x_tr.shape[1]

            w_initial = np.random.uniform(0,1,size=nb_features) 
            model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)

            model_ls.learn_weights(y_tr, x_tr)
            models.append(model_ls)

            tx_te, y_te = data_part[idx]['test']
            y_ = model_ls.predict(tx_te)
            acc_list[i].append(misc.accuracy(y_te, y_))
            
    return acc_list, nb_samples

In [8]:
cumul_eval = []

nb_runs = 4

for idx in range(nb_runs):
    print(idx + 1, 'out of', nb_runs)
    data_part = []

    nb_samples = []
    for x_, y_ in zip(x_split, y_split):
        train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
        data = {'train': train_data, 'test': test_data}
        data_part.append(data)
        nb_samples.append(len(test_data[1]))
        
    eval_result = train_and_eval(data_part)
    cumul_eval.append(eval_result)

1 out of 4
learning weights...
done.
learning weights...
done.
learning weights...
done.
2 out of 4
learning weights...
done.
learning weights...
done.
learning weights...
done.
3 out of 4
learning weights...
done.
learning weights...
done.
learning weights...
done.
4 out of 4
learning weights...
done.
learning weights...
done.
learning weights...
done.


In [9]:
cumul_eval

[([[0.8333500150135121, 0.8209956151663658, 0.8493245106148332]],
  [9991, 7754, 7254]),
 ([[0.8354519067160444, 0.8163528501418623, 0.8496002205679625]],
  [9991, 7754, 7254]),
 ([[0.8469622660394355, 0.8211245808614909, 0.8497380755445272]],
  [9991, 7754, 7254]),
 ([[0.8461615453908518, 0.8193190611297395, 0.8534601599117728]],
  [9991, 7754, 7254])]

In [29]:
# Determine the mean and std of data splitted on jet_num (and accumulated)
x_0 = []
x_1 = []
x_23 = []
avg_accs = []
for i in range(4):
    x_0.append(cumul_eval[i][0][0][0])
    x_1.append(cumul_eval[i][0][0][1])
    x_23.append(cumul_eval[i][0][0][2])
    avg_accs.append(np.dot(cumul_eval[i][0][0], nb_samples) / sum(nb_samples))

In [32]:
print(np.mean(x_0), np.std(x_0))
print(np.mean(x_1), np.std(x_1))
print(np.mean(x_23), np.std(x_23))
np.mean(avg_accs), np.std(avg_accs)

0.8404814332899608 0.006132253476429942
0.8194480268248647 0.001923708265434125
0.8505307416597738 0.0016978422851879889


(0.8368734749389977, 0.0029866224366019833)